In [1]:
from darli.modeling import Robot
from darli.modeling.functional import Functional
from darli.backend import PinocchioBackend, CasadiBackend
from robot_descriptions import z1_description
import numpy as np

In [2]:
robot = Robot(PinocchioBackend(z1_description.URDF_PATH))
robot.add_body({"ee": "link06", "mid": "link03"})

funct = Functional(CasadiBackend(z1_description.URDF_PATH))
funct.add_body({"ee": "link06", "mid": "link03"})

In [3]:
def compute_pin(robot: Robot):
    q, dq, ddq = np.random.rand(3, robot.nq)

    robot.update(q, dq, ddq)

    return {
        "com_pos": robot.com(),
        "kinetic": robot.energy().kinetic,
        "body1_pos": robot.body("ee").position,
        "body1_rot": robot.body("ee").rotation,
        "body2_pos": robot.body("mid").position,
        "body2_rot": robot.body("mid").rotation,
    }


def compute_casadi(funct: Functional):
    q, dq, ddq = np.random.rand(3, funct.nq)

    return {
        "com_pos": funct.com.position(q),
        "kinetic": funct.energy.kinetic(q, dq),
        "body1_pos": funct.body("ee").position(q),
        "body1_rot": funct.body("ee").rotation(q),
        "body2_pos": funct.body("mid").position(q),
        "body2_rot": funct.body("mid").rotation(q),
    }

In [4]:
compute_pin(robot)

/home/leo/github.com/lvjonok/darli/darli/backend/pinocchio.py:254: UserWarning: time variation of CoM jacobian is not implemented for pinocchio backend
  warnings.warn(


{'com_pos': CoM(position=array([-0.15285163, -0.02911732,  0.09897152]), velocity=array([-0.07693299, -0.08975777,  0.01485851]), acceleration=array([ 0.00740184, -0.09003013,  0.19983722]), jacobian=array([[ 2.91173198e-02, -7.88787496e-04, -1.05925595e-01,
         -2.40342880e-02, -1.22139369e-03,  5.37663184e-05],
        [-1.52851633e-01, -1.55877141e-04, -2.09326074e-02,
         -4.74956323e-03,  8.30745563e-03,  8.88199308e-05],
        [ 0.00000000e+00,  1.55597013e-01, -3.40885648e-02,
          1.05119376e-02,  9.30239462e-04, -2.15631089e-05]]), jacobian_dt=None),
 'kinetic': 0.07355631621019913,
 'body1_pos': array([-0.0128,  0.    ,  0.1605]),
 'body1_rot': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]),
 'body2_pos': array([-0.35  ,  0.    ,  0.1035]),
 'body2_rot': array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])}

In [5]:
compute_casadi(funct)

{'com_pos': DM([-0.162986, -0.0390951, 0.119837]),
 'kinetic': DM(0.0725384),
 'body1_pos': DM([-0.302153, -0.0618898, 0.018271]),
 'body1_rot': DM(
 [[-0.795204, 0.360226, 0.487736], 
  [0.1169, 0.880384, -0.459629], 
  [-0.594966, -0.308483, -0.742195]]),
 'body2_pos': DM([-0.276438, -0.0710894, 0.306055]),
 'body2_rot': DM(
 [[0.0511263, -0.249059, 0.967138], 
  [0.0131478, 0.968488, 0.248712], 
  [-0.998606, 0, 0.0527898]])}

In [6]:
%timeit compute_pin(robot)

97 µs ± 8.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%timeit compute_casadi(funct)

31.1 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
